In [1]:
require 'hdf5'
require 'nn'

In [2]:
-- Loading data
myFile = hdf5.open('../data/MM_data_cap.hdf5','r')
data = myFile:all()
input_matrix_train_cap = data['input_matrix_train_cap']
input_matrix_dev_cap = data['input_matrix_dev_cap']
input_matrix_test_cap = data['input_matrix_test_cap']
myFile:close()

In [8]:
nwords = input_matrix_train_cap:size(1)
train_input = torch.Tensor(nwords-1,10)
train_input:narrow(2,1,1):copy(input_matrix_train_cap:narrow(2,1,1):narrow(1,2,nwords-1))
train_input:narrow(2,2,9):copy(input_matrix_train_cap:narrow(2,2,9):narrow(1,1,nwords-1))
train_output = input_matrix_train_cap:narrow(2,16,1):narrow(1,2,nwords-1)

In [11]:
myFile = hdf5.open('../data/embeddings.hdf5','r')
data2 = myFile:all()
embeddings = data2['embeddings']
myFile:close()

In [13]:
LT = nn.LookupTable(400002,50)
LT.weight:narrow(1, 1, 400000):copy(embeddings)

In [15]:
model = nn.Sequential()
t1 = nn.ParallelTable()

t1_1 = nn.Sequential()
t1_1:add(LT)
t1_1:add(nn.View(-1,50))

t1_2 = nn.Identity()

t1:add(t1_1)
t1:add(t1_2)

model:add(t1)
model:add(nn.JoinTable(2))

model:add(nn.Linear(59,9))

In [ ]:
function compute_logscore(observations, i, model, C)
    local y = torch.zeros(C,C)
    local hot_1 = torch.zeros(C)
    for j = 1, C do
        hot_1:zero()
        hot_1[j] = 1
        y:narrow(1,j,1):copy(model:forward({observations[i]:view(1,1),hot_1:view(1,9)}))
    end
    return y
end

function viterbi(observations, compute_logscore, model, C)
    
    local y = torch.zeros(C,C)
    -- Formating tensors
    local initial = torch.zeros(C, 1)
    -- initial started with a start of sentence: <t>

    initial[{8,1}] = 1
    initial:log()

    -- number of classes
    local n = observations:size(1)
    local max_table = torch.Tensor(n, C)
    local backpointer_table = torch.Tensor(n, C)
    -- first timestep
    -- the initial most likely paths are the initial state distribution
    -- NOTE: another unnecessary Tensor allocation here
    local maxes, backpointers = (initial + compute_logscore(observations, 1, model, C)[8]):max(2)
    max_table[1] = maxes
    -- remaining timesteps ("forwarding" the maxes)
    for i=2,n do
        -- precompute edge scores
       
        y:copy(compute_logscore(observations, i, model, C))
        scores = y:transpose(1,2) + maxes:view(1, C):expand(C, C)

        -- compute new maxes (NOTE: another unnecessary Tensor allocation here)
        maxes, backpointers = scores:max(2)

        -- record
        max_table[i] = maxes
        backpointer_table[i] = backpointers
    end
    -- follow backpointers to recover max path
    local classes = torch.Tensor(n)
    maxes, classes[n] = maxes:max(1)
    for i=n,2,-1 do
        classes[i-1] = backpointer_table[{i, classes[i]}]
    end

    return classes
end

function train_model(train_input, train_output, model, criterion, din, nclass, eta, nEpochs, batchSize)
    -- Train the model with a mini batch SGD
    -- standard parameters are
    -- nEpochs = 1
    -- batchSize = 32
    -- eta = 0.01
    loss = torch.Tensor(nEpochs)

    -- To store the loss
    av_L = 0

    -- Memory allocation
    inputs_batch = torch.DoubleTensor(batchSize, din)
    gold_sequence = torch.DoubleTensor(batchSize)
    high_score_seq = torch.DoubleTensor(batchSize, nclass)
    df_do = torch.zeros(batchSize, nclass)

    for i = 1, nEpochs do
        -- timing the epoch
        timer = torch.Timer()
        av_L = 0
        
        -- mini batch loop
        for t = 1, train_input:size(1), batchSize do
            -- Mini batch data
            current_batch_size = math.min(batchSize,train_input:size(1)-t)
--             print('here1')
            
            inputs_batch:narrow(1,1,current_batch_size):copy(train_input:narrow(1,t,current_batch_size))
--             print('here2')
            
            gold_sequence:narrow(1,1,current_batch_size):copy(train_output:narrow(1,t,current_batch_size))
--             print('here3')
            
            -- reset gradients
            model:zeroGradParameters()
            --gradParameters:zero()

            -- Forward pass on a batch subsequence:
            high_score_seq:narrow(1,1,current_batch_size):copy(viterbi(inputs_batch:narrow(2,1,1), compute_logscore, model, nclass))
--             print('here4')
            
            -- TO CODE :
            df_do:zero()
            for ii = 1, high_score_seq:size(1) do
                if high_score_seq[ii] ~= gold_sequence_seq[ii] then
                    y = model:forward({inputs_batch:narrow(1,ii,1):narrow(2,1,1),inputs_batch:narrow(2,2,9)})
                    
                end
            end
--             print('here6')
            model:backward({inputs_batch:narrow(1,1,current_batch_size):narrow(2,1,1), inputs_batch:narrow(1,1,current_batch_size):narrow(2,2,9)}, 
                            df_do:narrow(1,1,current_batch_size))
--             print('here7')
            model:updateParameters(eta)
            
        end
            
        print('Epoch '..i..': '..timer:time().real)
        loss[i] = av_L/math.floor(train_input:size(1)/batchSize)
        print('Average Loss: '.. loss[i])
       
    end
    
    return loss
end